In [1]:
import os

In [2]:
%pwd
os.chdir(r"c:\Users\Admin\Desktop\text-summarizer\text-summarizer-project") 


In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    train_path: Path
    valid_path: Path
    test_path: Path
    tokenizer_name: str



In [4]:
import sys
sys.path.append(r"c:\Users\Admin\Desktop\text-summarizer\text-summarizer-project\src")


In [5]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        cfg = self.config.data_transformation

        # Must be inside the method
        create_directories([cfg.root_dir])

        return DataTransformationConfig(
            root_dir=Path(cfg.root_dir),
            train_path=Path(cfg.train_path),
            valid_path=Path(cfg.valid_path),
            test_path=Path(cfg.test_path),
            tokenizer_name=cfg.tokenizer_name
        )


In [7]:
import os
from  transformers import AutoTokenizer
from text_summarizer.logging import logger
from datasets import load_dataset, load_from_disk




c:\Users\Admin\anaconda3\envs\ts\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from datasets import load_dataset
from transformers import AutoTokenizer
import os
from text_summarizer.logging import logger
from datasets import load_from_disk
from transformers import AutoTokenizer

class DataTransformation:
    def __init__(self, config):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)

    def tokenize_function(self, examples):
        inputs = self.tokenizer(
            examples["dialogue"],
            max_length=512,
            truncation=True,
            padding="max_length"
        )
        labels = self.tokenizer(
            examples["summary"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
        inputs["labels"] = labels["input_ids"]
        return inputs

    def convert(self):
        print(f"Loading dataset from {self.config.data_path}")
        dataset = load_from_disk(self.config.data_path)  # load from .arrow files

        print("Tokenizing dataset...")
        tokenized_dataset = dataset.map(self.tokenize_function, batched=True)

        save_path = os.path.join(self.config.root_dir, "samsum_dataset_t5")
        print(f"Saving tokenized dataset to {save_path}")
        tokenized_dataset.save_to_disk(save_path)
        print("Saved successfully!")


In [9]:
from box import ConfigBox, Box
import yaml

# Load YAML file
with open("config/config.yaml", "r") as f:
    config_dict = yaml.safe_load(f)

# Convert to Box for attribute-style access
config = ConfigBox(config_dict)


In [10]:
train_path = config.data_ingestion.get("train_path", None)
if train_path is None:
    raise ValueError("train_path not found in config!")


In [11]:
from datasets import load_from_disk
dataset = load_from_disk("artifacts/data_ingestion/samsum_dataset")
dataset


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [12]:
try:
    config_manager = ConfigurationManager()
    data_transformation_config = config_manager.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

2025-12-01 15:40:52,172 - INFO - common - YAML file: config\config.yaml loaded successfully.
2025-12-01 15:40:52,213 - INFO - common - YAML file: params.yaml loaded successfully.
2025-12-01 15:40:52,219 - INFO - common - Directory created at: artifacts
2025-12-01 15:40:52,226 - INFO - common - Directory created at: artifacts/data_transformation


BoxKeyError: "'ConfigBox' object has no attribute 'train_path'"

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from transformers import AutoTokenizer
from datasets import load_from_disk
from text_summarizer.logging import logger
from text_summarizer.utils.common import create_directories
import os

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        create_directories([Path(self.config.root_dir)])
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)

    def tokenize_function(self, examples):
        inputs = self.tokenizer(
            examples["dialogue"],
            max_length=512,
            truncation=True,
            padding="max_length"
        )
        labels = self.tokenizer(
            examples["summary"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )["input_ids"]
        inputs["labels"] = labels
        return inputs

    def convert(self):
        logger.info(f"Loading dataset from: {self.config.data_path}")
        dataset = load_from_disk(str(self.config.data_path))
        logger.info("Tokenizing dataset (this may take a while)...")
        tokenized = dataset.map(self.tokenize_function, batched=True)
        save_path = Path(self.config.root_dir) / "samsum_dataset_t5"
        create_directories([save_path])
        tokenized.save_to_disk(str(save_path))
        logger.info(f"Saved tokenized dataset to: {save_path}")
        return save_path


: 

: 

: 

: 

: 